In [2]:
from dotenv import load_dotenv
import tiktoken
import openai
import pandas as pd
import os
import numpy as np
from openai.embeddings_utils import cosine_similarity, get_embedding

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_EMBEDDING_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

#### Read CSV using pandas

In [ ]:
input_datapath = "./data/recipes_onecol.csv"  
df = pd.read_csv(input_datapath)

### Using OpenAI model ada for embeddings

OpenAI offers one second-generation embedding model (denoted by -002 in the model ID) and 16 first-generation models (denoted by -001 in the model ID).
We recommend using text-embedding-ada-002 for nearly all use cases. It’s better, cheaper, and simpler to use. 

#### Initialize Embeddings model 

In [ ]:
# embedding model parameters
# encoding for text-embedding-ada-002
embedding_encoding = "cl100k_base"  
# the maximum for text-embedding-ada-002 is 8191
max_tokens = 8000 
# the number of reviews to embed
top_n = 1000
encoding = tiktoken.get_encoding(embedding_encoding)

#### Count nr of tokens in each row

In [ ]:
# skip Recipes that are too long to embed > max_tokens
df["n_tokens"] = df.Recipe.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)

#### Create embeddings

In [ ]:
# This may take a few minutes
df["embedding"] = df.Recipe.apply(lambda x: get_embedding(x, engine=OPENAI_EMBEDDING_MODEL_NAME))
# save the file
df.to_csv("./data/recipes_onecol_with_embeddings.csv")

#### Read embeddings

In [6]:
datafile_path = "./data/recipes_onecol_with_embeddings.csv"
df = pd.read_csv(datafile_path)
# convert the string representation of the embedding to a numpy array
df["embedding"] = df.embedding.apply(eval).apply(np.array)

#### Define a function to ask a question and get an answer from GPT-3.5

In [7]:
def ask_question(question):
    n=1
    question_embedding = get_embedding(
        question,
        engine=OPENAI_EMBEDDING_DEPLOYMENT_NAME
    )
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, question_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
    )
    answer =  ' '.join(results.Recipe.tolist()) 
    print("Answer:", answer)

#### Berries is similar to raspberries

In [9]:
ask_question("Chocolate and berries")

Answer: Recipe: Raspberry Sundaes with Chocolate Sauce and Roasted Almonds - Ingredients: 6 ounces bittersweet chocolate, finely chopped;3/4 cup water;2/3 cups sugar;2 tablespoons honey;3 tablespoons unsalted butter;Pinch of kosher salt;1 pint fresh raspberries (12 ounces);2 tablespoons sugar;2 tablespoons elderflower liqueur, such as St-Germain;2 tablespoons fresh lemon juice;2 pints vanilla ice cream;Chopped roasted unsalted almonds, for garnish- Instructions: To make the chocolate sauce, place a saucepan over medium-low heat, combine the chocolate, and water, vigorously stirring until completely melted and smooth. Let it come to a low simmer until small bubbles appear around the outside. Add sugar, honey and keep at a low simmer until thickened, about 3 minutes. Remove from heat and stir in butter and salt. Chocolate sauce will smooth out at is cools to room temperature. Store in an airtight container. In a bowl, gently toss together the raspberries, sugar, elderflower liqueur, and 

#### Bukgur instead of Bulgur

In [14]:
ask_question("Bukgur vegan recipe")

Answer: Recipe: Bulgur Veggie Burgers with Lime Mayonnaise - Ingredients: 1/2 cup chopped onion, divided;1 tablespoon olive oil plus additional for brushing;1/2 cup bulgur;1 cup water;1 cup canned pinto beans, rinsed and drained;1 1/2 tablespoon soy sauce;3/4 cup walnuts (2 1/2 ounces);2 garlic cloves, coarsely chopped;1/2 cup packed cilantro sprigs;3/4 teaspoon ground cumin;1/4 teaspoon cayenne;1/4 cup mayonnaise;1/4 teaspoon grated lime zest;1/2 teaspoon fresh lime juice;4 slices multi-grain bread,toasted;Equipment: a perforated grill sheet;Accompaniments: lettuce; sliced tomato- Instructions: Cook half of onion with 1/4 teaspoon salt in oil in a small heavy saucepan over medium heat, stirring occasionally, until golden, 5 to 7 minutes. Add bulgur and water and cook, covered, over low heat until water is absorbed, 15 to 18 minutes. Transfer to a bowl and stir in beans and soy sauce. Pulse bulgur mixture, walnuts, garlic, cilantro, cumin, cayenne, a rounded 1/4 teaspoon salt, 1/2 teas

In [18]:
ask_question("Fish no alcohol")

Answer: Recipe: Soft Fish Tacos - Ingredients: 1/4 cup mayonnaise;1/4 cup ketchup;1/4 cup crema mexicana*;1 cup all purpose flour;1 teaspoon fine sea salt;1/2 teaspoon ground pepper;1 cup dark beer, room temperature;13/4 pounds halibut, cut into 5x3/4-inch strips;1 lime, halved crosswise;12 white corn tortillas;Vegetable oil (for deep-frying);1 1/2 cups shredded red cabbage;2 large tomatoes, chopped;Lime wedges;Bottled hot pepper sauce- Instructions: For sauce: Mix all ingredients in bowl; season with salt and pepper.
Mix all ingredients in bowl; season with salt and pepper.
For batter and fish: Whisk flour, salt, and pepper in bowl; pour in beer, whisking until batter is smooth. Let stand 15 minutes. Sprinkle fish with salt and pepper. Squeeze some lime juice over each strip. Let stand 15 minutes. Mix fish into batter. Preheat oven to 200°F. Heat skillet over medium heat. Stack 2 tortillas. Sprinkle top with water. Place in skillet, wet side down. Heat 1 minute. Sprinkle top with wate

In [17]:
ask_question("Fish with alcohol")

Answer: Recipe: Soft Fish Tacos - Ingredients: 1/4 cup mayonnaise;1/4 cup ketchup;1/4 cup crema mexicana*;1 cup all purpose flour;1 teaspoon fine sea salt;1/2 teaspoon ground pepper;1 cup dark beer, room temperature;13/4 pounds halibut, cut into 5x3/4-inch strips;1 lime, halved crosswise;12 white corn tortillas;Vegetable oil (for deep-frying);1 1/2 cups shredded red cabbage;2 large tomatoes, chopped;Lime wedges;Bottled hot pepper sauce- Instructions: For sauce: Mix all ingredients in bowl; season with salt and pepper.
Mix all ingredients in bowl; season with salt and pepper.
For batter and fish: Whisk flour, salt, and pepper in bowl; pour in beer, whisking until batter is smooth. Let stand 15 minutes. Sprinkle fish with salt and pepper. Squeeze some lime juice over each strip. Let stand 15 minutes. Mix fish into batter. Preheat oven to 200°F. Heat skillet over medium heat. Stack 2 tortillas. Sprinkle top with water. Place in skillet, wet side down. Heat 1 minute. Sprinkle top with wate